# Decision Trees

# Gradient Boosting

# XGBoost


## XGBoost Features



x

## System Features

- Paralelización
- Cómputo Distribuido
- Out-of-Core Computing
- Cache: Optimización de est

## Algorithm Features

-

## Por qué usar XGBoost?
